# Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
from citipy import citipy 
from datetime import datetime

# Import the API keys.
from config import g_key
from config import weather_api_key

# configure gmaps to use Google API key
gmaps.configure(api_key=g_key)



In [2]:
#Import CSV
weather_data_to_load = "weather_data/WeatherPy_challenge.csv"

In [3]:
# Read the weather data file and store it in a pandas DataFrame.
weather_data_df = pd.read_csv(weather_data_to_load)
weather_data_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Sibolga,ID,2020-02-23 21:25:27,1.74,98.78,70.74,89,0,4.03,clear sky,0.0,0.0
1,1,Punta Arenas,CL,2020-02-23 21:24:05,-53.15,-70.92,55.40,62,75,31.09,broken clouds,0.0,0.0
2,2,Cidreira,BR,2020-02-23 21:25:27,-30.18,-50.21,71.91,73,1,17.29,clear sky,0.0,0.0
3,3,Port Hardy,CA,2020-02-23 21:25:27,50.70,-127.42,41.00,93,90,2.24,light rain,0.0,0.0
4,4,Port Alfred,ZA,2020-02-23 21:25:28,-33.59,26.89,60.01,83,50,2.01,scattered clouds,0.0,0.0
5,5,Yellowknife,CA,2020-02-23 21:29:25,62.46,-114.35,-2.99,63,20,6.93,few clouds,0.0,0.0
6,6,Utica,US,2020-02-23 21:30:11,43.10,-75.23,48.00,33,1,9.17,clear sky,0.0,0.0
7,7,Sabzevar,IR,2020-02-23 21:33:43,36.21,57.68,50.31,50,0,3.31,clear sky,0.0,0.0
8,8,Avarua,CK,2020-02-23 21:33:44,-21.21,-159.78,78.80,94,100,4.70,moderate rain,0.0,0.0
9,9,Saskylakh,RU,2020-02-23 21:33:44,71.92,114.08,5.74,91,100,8.63,overcast clouds,0.0,0.0


In [4]:
#Remove unnamed column
del weather_data_df["Unnamed: 0"]

In [5]:
weather_data_df
weather_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Sibolga,ID,2020-02-23 21:25:27,1.74,98.78,70.74,89,0,4.03,clear sky,0.0,0.0
1,Punta Arenas,CL,2020-02-23 21:24:05,-53.15,-70.92,55.40,62,75,31.09,broken clouds,0.0,0.0
2,Cidreira,BR,2020-02-23 21:25:27,-30.18,-50.21,71.91,73,1,17.29,clear sky,0.0,0.0
3,Port Hardy,CA,2020-02-23 21:25:27,50.70,-127.42,41.00,93,90,2.24,light rain,0.0,0.0
4,Port Alfred,ZA,2020-02-23 21:25:28,-33.59,26.89,60.01,83,50,2.01,scattered clouds,0.0,0.0


In [ ]:
# Ask the customer to add a minimum, maximum temperature value and if they want rainfall or snow.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

In [7]:
# Filter rain and snow dataset
if rain == "no" and snow == "no":
    filtered_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                       (weather_data_df["Max Temp"] >= min_temp) &
                                       (weather_data_df["Rain Inches (last 3 hrs)"] == 0) &
                                         (weather_data_df["Snow Inches (last 3 hrs)"] == 0)]
    
elif rain == "no" and snow == "yes":
    filtered_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                       (weather_data_df["Max Temp"] >= min_temp) &
                                       (weather_data_df["Rain Inches (last 3 hrs)"] == 0) &
                                         (weather_data_df["Snow Inches (last 3 hrs)"] > 0)]
    
elif rain == "yes" and snow == "yes":
    filtered_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                       (weather_data_df["Max Temp"] >= min_temp) &
                                       (weather_data_df["Rain Inches (last 3 hrs)"] > 0) &
                                         (weather_data_df["Snow Inches (last 3 hrs)"] > 0)]
    
elif rain == "yes" and snow == "no":
    filtered_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                       (weather_data_df["Max Temp"] >= min_temp) &
                                       (weather_data_df["Rain Inches (last 3 hrs)"] > 0) &
                                        (weather_data_df["Snow Inches (last 3 hrs)"] == 0)]
    
filtered_cities_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Sibolga,ID,2020-02-23 21:25:27,1.74,98.78,70.74,89,0,4.03,clear sky,0.0,0.0
1,Punta Arenas,CL,2020-02-23 21:24:05,-53.15,-70.92,55.40,62,75,31.09,broken clouds,0.0,0.0
2,Cidreira,BR,2020-02-23 21:25:27,-30.18,-50.21,71.91,73,1,17.29,clear sky,0.0,0.0
3,Port Hardy,CA,2020-02-23 21:25:27,50.70,-127.42,41.00,93,90,2.24,light rain,0.0,0.0
4,Port Alfred,ZA,2020-02-23 21:25:28,-33.59,26.89,60.01,83,50,2.01,scattered clouds,0.0,0.0
6,Utica,US,2020-02-23 21:30:11,43.10,-75.23,48.00,33,1,9.17,clear sky,0.0,0.0
7,Sabzevar,IR,2020-02-23 21:33:43,36.21,57.68,50.31,50,0,3.31,clear sky,0.0,0.0
8,Avarua,CK,2020-02-23 21:33:44,-21.21,-159.78,78.80,94,100,4.70,moderate rain,0.0,0.0
14,Marawi,PH,2020-02-23 21:33:45,8.00,124.29,73.00,87,42,5.99,scattered clouds,0.0,0.0
15,Taolanaro,PH,2020-02-23 21:33:45,8.00,124.29,73.00,87,42,5.99,scattered clouds,0.0,0.0


In [8]:
filtered_cities_df.count()

City                        315
Country                     312
Date                        315
Lat                         315
Lng                         315
Max Temp                    315
Humidity                    315
Cloudiness                  315
Wind Speed                  315
Current Description         315
Rain Inches (last 3 hrs)    315
Snow Inches (last 3 hrs)    315
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sibolga,ID,70.74,clear sky,1.74,98.78,
1,Punta Arenas,CL,55.40,broken clouds,-53.15,-70.92,
2,Cidreira,BR,71.91,clear sky,-30.18,-50.21,
3,Port Hardy,CA,41.00,light rain,50.70,-127.42,
4,Port Alfred,ZA,60.01,scattered clouds,-33.59,26.89,
6,Utica,US,48.00,clear sky,43.10,-75.23,
7,Sabzevar,IR,50.31,clear sky,36.21,57.68,
8,Avarua,CK,78.80,moderate rain,-21.21,-159.78,
14,Marawi,PH,73.00,scattered clouds,8.00,124.29,
15,Taolanaro,PH,73.00,scattered clouds,8.00,124.29,


In [12]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("success")
    except (IndexError) as error:
        print("Hotel not found... skipping.")
    

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
Hotel not found... skipping.
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
Hotel not found... skipping.
success
success
success
success
success
success
success
success
success
success
success
success
success
Hotel not found... skipping.
Hotel not found... skipping.
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
succ

In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sibolga,ID,70.74,clear sky,1.74,98.78,Hotel Syariah CN Darussalam
1,Punta Arenas,CL,55.40,broken clouds,-53.15,-70.92,Hotel Dreams Del Estrecho
2,Cidreira,BR,71.91,clear sky,-30.18,-50.21,Novomar Pousada e Turismo Ltda
3,Port Hardy,CA,41.00,light rain,50.70,-127.42,Airport Inn
4,Port Alfred,ZA,60.01,scattered clouds,-33.59,26.89,The Halyards Hotel
6,Utica,US,48.00,clear sky,43.10,-75.23,Days Inn by Wyndham Utica
7,Sabzevar,IR,50.31,clear sky,36.21,57.68,پارکینگ وسایل سنگین جنوب
8,Avarua,CK,78.80,moderate rain,-21.21,-159.78,Paradise Inn
14,Marawi,PH,73.00,scattered clouds,8.00,124.29,Derogongan Residence
15,Taolanaro,PH,73.00,scattered clouds,8.00,124.29,Derogongan Residence


In [24]:
# Using the template to information to the markers for the cities.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [27]:
#Save as CSV
output_data_file = "weather_data/WeatherPy_vacation.csv"

# Export the full_data into a CSV.
hotel_df.to_csv(output_data_file)

In [28]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add marker spots for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
#Four cities for vacation search
Lata_df=hotel_df.loc[hotel_df["City"]=="Lata"]
Lata_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
501,Lata,PT,62.01,clear sky,40.16,-8.33,Casa Maquia


In [41]:
#Four cities for vacation search
Montpellier_df=hotel_df.loc[hotel_df["City"]=="Montpellier"]
Montpellier_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
136,Montpellier,FR,60.8,clear sky,43.61,3.88,Hôtel des Arts Comédie


In [39]:
#Four cities for vacation search
Arroyo_df=hotel_df.loc[hotel_df["City"]=="Arroyo"]
Arroyo_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
305,Arroyo,ES,46.0,clear sky,41.61,-4.8,Hotel El Jardín de la Abadía


In [38]:
#Four cities for vacation search
Ischia_df=hotel_df.loc[hotel_df["City"]=="Ischia"]
Ischia_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
388,Ischia,IT,57.0,scattered clouds,40.73,13.95,Hotel Ulisse


In [43]:
#Create directions with latitude-longitude
Lata = (40.16, -8.33)
Arroyo = (41.61, -4.8)
Montpellier= (43.61, 3.88)
Ischia = (40.73, 13.95)

fig = gmaps.figure()
Lata_to_Ischia=gmaps.directions_layer(Lata, Ischia, waypoints=[Arroyo, Montpellier], travel_mode='DRIVING')
fig.add_layer(Lata_to_Ischia)
fig

Figure(layout=FigureLayout(height='420px'))

In [44]:
#New DataFrame for traveling cities
four_cities_df=pd.concat([Lata_df, Arroyo_df, Montpellier_df, Ischia_df], ignore_index=True)
four_cities_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lata,PT,62.01,clear sky,40.16,-8.33,Casa Maquia
1,Arroyo,ES,46.00,clear sky,41.61,-4.80,Hotel El Jardín de la Abadía
2,Montpellier,FR,60.80,clear sky,43.61,3.88,Hôtel des Arts Comédie
3,Ischia,IT,57.00,scattered clouds,40.73,13.95,Hotel Ulisse


In [45]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt>{Country}</dd>
<dt>Current Weather</dt>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

In [46]:
# Add a marker for each city.

hotel_info = [info_box_template.format(**row) for index, row in four_cities_df.iterrows()]
                                     
locations = four_cities_df[["Lat", "Lng"]]
max_temp = four_cities_df["Max Temp"]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))